In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm, colors
from pprint import pprint
from scipy.ndimage import gaussian_filter1d
from IPython.display import display

# brainbox / iblatlas / ONE 관련
from brainbox.io.one import SessionLoader, SpikeSortingLoader
from brainbox.singlecell import bin_spikes
from brainbox.ephys_plots import plot_brain_regions
from iblatlas.atlas import AllenAtlas
from one.api import ONE

import os
import sys

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))

def add_module_paths(base, *rel_paths):
    for rel_path in rel_paths:
        sys.path.append(os.path.join(base, *rel_path))

add_module_paths(BASE_DIR,
    ['func'],               # func 바로 아래 함수들
    ['func', 'compute'],
    ['func', 'info'],
    ['func', 'plot']
)

from compute_raster import compute_raster
from sub_func import save_file
from get_task_type import get_task_type

SAVE_PATH = r"C:\Users\miasc\SCH\shinlab\ChaeHyeon_Seong\dataset_v2"


Current working directory: c:\Users\miasc\SCH\shinlab\IBL\VISp_PSTH\main
Python path: ['c:\\Users\\miasc\\anaconda3\\envs\\iblenv\\python39.zip', 'c:\\Users\\miasc\\anaconda3\\envs\\iblenv\\DLLs', 'c:\\Users\\miasc\\anaconda3\\envs\\iblenv\\lib', 'c:\\Users\\miasc\\anaconda3\\envs\\iblenv', '', 'c:\\Users\\miasc\\anaconda3\\envs\\iblenv\\lib\\site-packages', 'c:\\Users\\miasc\\anaconda3\\envs\\iblenv\\lib\\site-packages\\win32', 'c:\\Users\\miasc\\anaconda3\\envs\\iblenv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\miasc\\anaconda3\\envs\\iblenv\\lib\\site-packages\\Pythonwin', '././fucn/', '././fucn/', '././fucn/']


In [ ]:
one = ONE()

brain_acronym = 'VISp'

# -----------------------------------------------------------------
# < Session 별로 정보 저장 >

# - 각 세션에 대해, 세션 폴더를 생성. 폴더명은 Session ID

# - 세션 폴더 내에, 저장하는 정보:
#  1) session_path.txt : 해당 세션의 ONE 경로 정보
#  2) task_type.txt : 해당 세션의 task type
#  3) trials_info.csv : 해당 세션의 trials 정보
#     (stimOn_times, contrastLeft, contrastRight, choice, feedbackType, response_times)
#     (task_type도 포함)

    # < Probe 별로 정보 저장 >

    # - 세션 폴더 내에, 각 프로브 별로 폴더를 생성. 폴더명은 probe label

    # - 각 프로브 폴더 내에, 저장하는 정보:
    #  1) mean_fr_session.npy : 해당 프로브의 전체 recording에서의 평균 firing rate
    #  2) neuron_info.csv : 해당 프로브의 뉴런 정보 (spike waveform, firing rate 등)
    #  3) psth.npy : 해당 프로브의 PSTH 정보 (spike raster)
    #  4) time_bins.npy : PSTH의 시간 bin 정보

# -----------------------------------------------------------------

# << Session ID list >>
sessions = list(one.search(atlas_acronym=brain_acronym, query_type='remote'))

for i in [44, 45, 68, 68, 68]: sessions.pop(i)
print(f"Found {len(sessions)} sessions for region: {brain_acronym}")

# -----------------------------------------------------------------
# {1} : Session 별 정보 저장
# -----------------------------------------------------------------
i = 1 # Session 번호 초기화

for eid in sessions:

    if i == 10: break # i = n : 원하는 세션 수 n 만큼 저장 후 종료

    # --- Session ID (EID) ---
    print(f"\n=== [Session:{i} {eid}] ==="); i += 1

    # -----------------------------------------------------------------
    # {1}-(1) : Dataset 내에 Session 폴더 생성 (폴더명: Session ID)
    # -----------------------------------------------------------------
    SAVE_SESSION_PATH = os.path.join(SAVE_PATH, eid); os.makedirs(SAVE_SESSION_PATH, exist_ok=True)

    # [ Save 1 ] : session_path.txt - 해당 세션의 ONE 경로 정보 저장
    
    with open(os.path.join(SAVE_SESSION_PATH, "session_path.txt"), "w", encoding="utf-8") as f:
        f.write(str(one.eid2path(eid)))

    # -----------------------------------------------------------------
    # {1}-(2) : Task type (Basic task or Full task) 판별
    # -----------------------------------------------------------------
    task_type = get_task_type(trials_df, print_info=True)

    # [ Save 2 ] : task_type.txt - 해당 세션의 task type 저장
    with open(os.path.join(SAVE_SESSION_PATH, "task_type.txt"), "w", encoding="utf-8") as f:
        f.write(task_type)

    # -----------------------------------------------------------------
    # {1}-(3) : Trials 정보 저장
    # -----------------------------------------------------------------
    sl = SessionLoader(eid=eid, one=one)
    sl.load_trials()
    trials_df = sl.trials
    trials_df = trials_df[['stimOn_times', 'contrastLeft', 'contrastRight',
                                    'choice', 'feedbackType', 'response_times']]
    trials_df['task_type'] = task_type

    # [ Save 3 ] : Trials 정보 저장
    save_file(trials_df, save_path=SAVE_SESSION_PATH, save_title="trials_info")


# << Probe ID list >>

    pids, labels = one.eid2pid(eid)
    if len(pids) == 0: 
        print(f" - No probe data found in session {eid}, skip."); continue # No probe data found
    
    # -----------------------------------------------------------------
    # {2} : Probe 별 정보 저장
    # -----------------------------------------------------------------

    for pid, label in zip(pids, labels):

        # --- Prove ID (PID) ---
        print(f"   ===> [Probe: {pid} ({label})]")

        # -----------------------------------------------------------------
        # {2}-(1) : Session 폴더 내에 Prove 별 폴더 생성 (폴더명: probe label)
        # -----------------------------------------------------------------
        SAVE_PROBE_PATH = os.path.join(SAVE_SESSION_PATH, label); os.makedirs(SAVE_PROBE_PATH, exist_ok=True)


        # -------------------------------------------------------------
        # {2}-(2) : Spike Sorting 정보 (spikes, clusters, channels)
        # -------------------------------------------------------------
        ssl = SpikeSortingLoader(one=one, pid=pid, atlas=AllenAtlas())
        spikes, clusters, channels = ssl.load_spike_sorting()
        clusters_from_ssl = ssl.merge_clusters(spikes, clusters, channels)
        clusters_df = clusters_from_ssl.to_df()


        # -------------------------------------------------------------
        # {2}-(3) : Neuron별 mean Firing Rate 계산 (전체 recording 구간에서의 평균 FR)
        # -------------------------------------------------------------
    
        recording_duration = spikes.times[-1] - spikes.times[0] # 전체 recording의 시간 (마지막 - 첫 spike 시간) (초 단위)
        # 각 unit에 대해 spike 개수를 세고, 전체 recording 기간(마지막 - 첫 spike 시간)으로 나눔
        unique_units = np.unique(spikes.clusters)
        mean_fr_dict = {unit: np.sum(spikes.clusters == unit) / recording_duration 
                        for unit in unique_units}
        clusters_df['meanFR'] = clusters_df.index.map(lambda x: mean_fr_dict.get(x, np.nan))

        # [ Save 4 ] : meanFR 정보 저장 (Method 1용)
        np.save(os.path.join(SAVE_PROBE_PATH, "meanFR.npy"), clusters_df['meanFR'].values)

        # -------------------------------------------------------------
        # {2}-(4) : Spike Type 정보 (peakToTrough, fast-spiking vs regular-spiking)
        # -------------------------------------------------------------
        clusters_from_obj = one.load_object(eid, 'clusters', collection=f'alf/{label}/pykilosort')

        # peakToTrough 값이 있는 경우, spikeType을 계산하여 pekToTrough 값과 함께 clusters_df에 추가
        if 'peakToTrough' in clusters_from_obj:
            peak2trough = clusters_from_obj['peakToTrough']

            # spikeType 계산: peakToTrough 값이 임계값(0.5ms)보다 작으면 'fast-spiking', 그렇지 않으면 'regular-spiking'
            clusters_df['peakToTrough'] = peak2trough
            threshold = 0.5  # 임계값 (ms)
            clusters_df['spikeType'] = clusters_df['peakToTrough'].abs().apply(
                lambda x: 'fast-spiking' if x < threshold else 'regular-spiking'
            )

        else: # 만약 없다면, NaN으로 처리, spikeType를 'unknown'으로 설정
            print(f" - No peakToTrough data found for probe {pid}, setting to NaN.")
            peak2trough = np.full(len(clusters_df), np.nan)
            clusters_df['peakToTrough'] = peak2trough # NaN으로 설정
            clusters_df['spikeType'] = 'unknown'  # spikeType을 'unknown'으로 설정  


        # [ Save 5 ] : neuron_info.csv - cluster_df 정보 저장 (meanFR, peakToTrough, spikeType 등 포함)
        save_file(clusters_df, save_path=SAVE_PROBE_PATH, save_title="neuron_info")

        # -------------------------------------------------------------
        # {2}-(4) PSTH 계산 (trial window 기반은 그대로 두되, Method 2, 3 계산은 그대로 수행)
        # -------------------------------------------------------------
    
        pre_time = 2.0
        post_time = 4.0
        bin_size = 0.001  # 1ms
        # psth의 시간 배열 (-2 ~ +4초)
        # (참고: 이 PSTH는 trial-locked window로 계산됨)
        # times = np.arange(-pre_time, post_time, bin_size)  <- 이미 compute_raster 내부에서 사용됨

        spike_raster, time_bins = compute_raster(spikes, 
                                                  np.where((clusters_df.index.values)[clusters_df.index.isin(unique_units)])[0],
                                                  events=sl.trials['stimOn_times'].values,
                                                  pre_time=pre_time,
                                                  post_time=post_time,
                                                  bin_size=bin_size)
        
        # spike_raster는 (num_units, num_bins) 형태의 2D 배열
        # time_bins는 (num_bins,) 형태의 1D 배열
        # spike_raster의 각 unit에 대해, firing rate를 계산
        
        # [ Save 6 ] : PSTH 정보 저장 - spike_raster와 time_bins
        save_file(spike_raster, save_path=SAVE_PROBE_PATH, save_title="psth")
        save_file(time_bins, save_path=SAVE_PROBE_PATH, save_title="time_bins")

print("\n\nAll done!")
